<a href="https://colab.research.google.com/github/mehrshad-sdtn/DeepLearning/blob/master/PyTorch/1_Pytorch_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: import all the necessary packages for common pytorch programs

import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


In [ ]:
class CNN(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels= in_channels, out_channels= 8, kernel_size= (3, 3), stride= (1, 1), padding= (1, 1))
        self.pool = nn.MaxPool2d(kernel_size= (2, 2), stride= (2, 2))
        self.conv2 = nn.Conv2d(in_channels= 8, out_channels= 16, kernel_size= (3, 3), stride= (1, 1), padding= (1, 1))
        self.fc1 = nn.Linear(16 * 7 * 7, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)

        return x


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
# hyperparameters
input_size = 784
num_classes = 10
learning_rate = 0.0001
batch_size = 64
num_epochs = 10

In [ ]:
# load Data
train_dataset = datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# initialize network
model = CNN(in_channels=1, num_classes= num_classes).to(device)

In [ ]:
# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# check accurcy on training and test
def check_accuracy(loader, model):
  if loader.dataset.train:
    print('Checking accuracy on training data:')
  else:
    print('Checking accuracy on test data')

  num_correct = 0
  num_samples = 0
  model.eval()
  with torch.no_grad():
    for x, y in loader:
      x = x.to(device= device)
      y = y.to(device= device)

      scores = model(x) # 64, 10
      _, predictions = scores.max(1)
      num_correct += (predictions == y).sum()
      num_samples += predictions.size(0)

    print(f" {float(num_correct)/float(num_samples)*100:.2f}%")
    model.train()


# train
for epoch in range(num_epochs):
  for batch_idx, (data, targets) in enumerate(train_loader):
    # data shaping
    data = data.to(device)
    targets = targets.to(device)

    # forward
    scores = model(data)
    loss = criterion(scores, targets)

    # backward
    optimizer.zero_grad()
    loss.backward()

    # gradient descent
    optimizer.step()

  print(f"Epoch {epoch+1}/{num_epochs}: loss {loss}")
  check_accuracy(train_loader, model)
  check_accuracy(test_loader, model)




Epoch 1/10: loss 0.5718998908996582
Checking accuracy on training data:
 87.96%
Checking accuracy on test data
 88.93%
Epoch 2/10: loss 0.45791107416152954
Checking accuracy on training data:
 91.42%
Checking accuracy on test data
 92.07%
Epoch 3/10: loss 0.17000322043895721
Checking accuracy on training data:
 93.38%
Checking accuracy on test data
 93.85%
Epoch 4/10: loss 0.08540171384811401
Checking accuracy on training data:
 94.36%
Checking accuracy on test data
 94.82%
Epoch 5/10: loss 0.1383964568376541
Checking accuracy on training data:
 95.18%
Checking accuracy on test data
 95.46%
Epoch 6/10: loss 0.1378350853919983
Checking accuracy on training data:
 95.81%
Checking accuracy on test data
 96.05%
Epoch 7/10: loss 0.06640152633190155
Checking accuracy on training data:
 96.27%
Checking accuracy on test data
 96.53%
Epoch 8/10: loss 0.05078206583857536
Checking accuracy on training data:
 96.63%
Checking accuracy on test data
 96.85%
Epoch 9/10: loss 0.054490819573402405
Check